# The 500 Greatest Albums of All Time

In 2003, the Rolling Stones magazinde published an article about a selection of the 500 greatest albums of the history. This database will serve as an exercise to practice dataset cleaning and to complete information using APIs from Wikipedia and Spotify.



## Dataset cleaning and cross correlation with API data

I obtained the top 500 albums dataset from the Kaggle website, using the following link:

https://www.kaggle.com/notgibs/500-greatest-albums-of-all-time-rolling-stone?select=albumlist.csv

To procede with the data cleaning we begin iporting the neccesaray library and functions. In enrich_functions.py, that we import in this notebook, we have several user made functions to work better with the selected APIs.


In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from enrich_functions import *
import re

## Data cleaning

We begin importing the dataset and showing its header, to have a look at how it is organized.

In [2]:
top500_raw = pd.read_csv("../dataset/albumlist.csv", index_col=0, encoding = "ISO-8859-1")
top500_raw.head()

Year                                  Album          Artist  \
Number                                                                
1       1967  Sgt. Pepper's Lonely Hearts Club Band     The Beatles   
2       1966                             Pet Sounds  The Beach Boys   
3       1966                               Revolver     The Beatles   
4       1965                   Highway 61 Revisited       Bob Dylan   
5       1965                            Rubber Soul     The Beatles   

            Genre                       Subgenre  
Number                                            
1            Rock  Rock & Roll, Psychedelic Rock  
2            Rock     Pop Rock, Psychedelic Rock  
3            Rock     Psychedelic Rock, Pop Rock  
4            Rock          Folk Rock, Blues Rock  
5       Rock, Pop                       Pop Rock

We begin by changing the column names to lowercase. Even though we can work with capitalized words, this is just a convention.

In [3]:
top500_raw.columns = map(str.lower, top500_raw.columns)

Now let's do a simple cleaning of bad characters in the genre and sub genre columns. 

There are albums that combine genre and subgenres, so we create for each cell a list with the different categories.

In [4]:
clean = lambda x: x.replace("Ê"," ").split(", ")
top500_raw["genre"]=top500_raw["genre"].apply(clean)
top500_raw["subgenre"]=top500_raw["subgenre"].apply(clean)

We need to do some treatement on the albums column, as there are some of them whose same is not the same as the one listed in the Spotify catalog. They also don't admit "'", so we eliminate them. 

To conduct this cleaning we create a dictionary in a function to replace bad names with RegEx. We apply this function to the "album" column using Panda's apply.

In [5]:
def wrong(x):
    dic={'The Beatles ("The White Album")':"The White Album",
          'Sign "Peace" the Times':'Sign "O" the Times',
          'The Band ("The Brown Album")':'The Band',
          'Back to Mono (1958-1969)':'Back To Mono',
          '[Led Zeppelin IV]':'Led Zeppelin IV',
          'Saturday Night Fever: The Original Movie Sound Track':'Saturday Night Fever',
          "The B 52's / Play Loud":"The B 52's",
          "The Very Best of Linda Ronstadt":"The Best of Linda Ronstadt",
          "The Anthology: 1961-1977": "The Very Best of Curtis Mayfield",
          'Blues Breakers With Eric Clapton ("The Beano Album")':"Bluesbreakers",
          "Bo Diddley / Go Bo Diddley":"Go Bo Diddley",
          "Can't Buy a Thrill":"Cant Buy a Thrill",
          'Metallica ("The Black Album")':"Metallica",
          "Trans Europa Express":"Trans-Europe Express",
          "'":"",
          "Honky Chteau":"Honky Chateau",
          "Cheap Trick at Budokan":"At Budokan"
          }
    xr=x
    for wrong,right in dic.items():
        xr=xr.replace(wrong,right)
    return xr

top500_raw["album"]=top500_raw["album"].apply(wrong)

In [6]:
top500_raw.drop([178,196,198,214,245,278,289,346,363,377,400,421,449,170,459],inplace=True)

## Completing the information using the Spotify API

Using the credentials obtained from Spotify, we are going to complete the information from each album. We are going to do by searching the unique album ID for each input. With this ID, we are going to collect all the tracks for each album, and from each track, we are going to download the musical features (danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence and tempo). I am going to compute the median of all tracks in each album to obtain the median album features.

To do so, we load the credential for the Spotify API, and import the `Spotipy` library, used to manage the API requests. In addition, in `enrich_functions.py`, there are stored functions to retrive the desired data, that were made for this specific project.

First of all, we initialize the Spotify API mangament system in python with `Spotipy` and loading the ID and secret token:

In [7]:
load_dotenv()
cid = os.getenv("SPOTIFY_ID")
secret = os.getenv("SPOTIFY_SECRET")
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
spotify

Now, to retrive the mentioned musical information from each album, we define a lambda function that uses the user made function `get_album_features_median`, whose inputs are the Spotify API client (`spotify`) and the string of the album name. 

In [8]:
features_median = lambda x: get_album_features_median(spotify,x)


We apply this function to the column album, that stores the name of each album.

In [9]:
top500_raw["features"]=top500_raw["album"].apply(features_median)

Lastly, this funtion output is a dictionary with the median values of danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence and tempo for each album.

For working better with this data, we add a column for each feature using the Pandas command `concat`.

In [10]:
top500_raw=pd.concat([top500_raw, top500_raw["features"].apply(pd.Series)], axis=1, sort=False)

## Completing the information using the Wikipedia API

Now we are going to complete even more the top 500 album dataset by looking at the Wikipedia web page monthly reads for each album and each artists in Spain. This will allow us to compare the curiosity about albums and artist information with the position in the top.

To do so, we defined a function in `enrich_functions.py` called `get_wiki_views`. This function output is the median monthly visits for a certain page in a range of time. The inputs are the Wikipedia language ("es" for Spain), the name of the Wikipedia article, the initial and ending year, month and days to compute the median monthly reads (we are goint to check it between 2019-10-01 and 2020-10-01).

We create a lambda funcion so the only input is the dersired album or artist, and we create a column with eac information using Pandas `apply`.

In [11]:
wiki= lambda x: get_wiki_views("es",x,2019,10,1,2020,10,1)
top500_raw["wiki_views_album"]=top500_raw["album"].apply(wiki)
top500_raw["wiki_views_artist"]=top500_raw["artist"].apply(wiki)

/home/ordovas/anaconda3/envs/ironhack/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Finally, we store the clean dataset in a new file called `albumlist_clean.csv`. We are going to load this cleaned file in the data-analysis notebook to conduct the analysis of the dataset that we prepared in this notebook.

In [12]:
top500_raw.to_csv('../dataset/albumlist_clean.csv')